Copyright (c) 2024-2025 Ken Barker

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Compare Geodesic and Great Circle Initial Azimuths

Compare the intial azimuth of geodesic paths to great circles.  
It calculates great circle intial azimuths using geodetic and parametric coordinates.

Performs calculations and compares the results with data from Charles Karney's [Test data for geodesics](https://geographiclib.sourceforge.io/C++/doc/geodesic.html#testgeod).

The contents of the data file are as follows:

- 0-100000 entries randomly distributed
- 100000-150000 entries which are nearly antipodal
- 150000-200000 entries with short distances
- 200000-250000 entries with one end near a pole
- 250000-300000 entries with both ends near opposite poles
- 300000-350000 entries which are nearly meridional
- 350000-400000 entries which are nearly equatorial
- 400000-450000 entries running between vertices (α1 = α2 = 90°)
- 450000-500000 entries ending close to vertices

In [ ]:
import numpy as np
import polars as pl
from scipy import stats
from enum import Enum

from via_angle import Angle, Degrees
from via_sphere import calculate_gc_azimuth
from via_units import Metres
from via_ellipsoid import Ellipsoid

## Read the GeodTest.dat file into a polars LazyFrame

In [ ]:
# The colums of the GeodTest.dat file
class Column(Enum):
    latitude_1   = 0
    longitude_1  = 1
    azimuth_1    = 2
    latitude_2   = 3
    longitude_2  = 4
    azimuth_2    = 5
    distance_m   = 6
    distance_deg = 7
    m12          = 8
    area         = 9

# Read the geodesic test data file into a polars LazyFrame: lf
# Select the random test data entries
size = 100000
pathname = 'https://sourceforge.net/projects/geographiclib/files/testdata/GeodTest.dat.gz/download'
lf = pl.scan_csv(pathname, separator=' ', has_header=False).select(
    ['column_1', 'column_2', 'column_3', 'column_4', 'column_5']
).head(size).collect()
lf.schema

### Calculate great circle initial azimuths

In [ ]:
%%time
delta_gc = np.empty(size)
delta_aux = np.empty(size)

i = 0
for row in lf.iter_rows():
    # calculate the great circle initial azimuth
    lat_1 = Angle(Degrees(row[Column.latitude_1.value]))
    lat_2 = Angle(Degrees(row[Column.latitude_2.value]))
    delta_long = Angle(Degrees(row[Column.longitude_2.value]))
    azim_gc = calculate_gc_azimuth(lat_1, lat_2, delta_long).to_degrees().v()
    delta_gc[i] = np.abs(azim_gc - row[Column.azimuth_1.value])
    
    # calculate the auxillary sphere great circle initial azimuth
    beta1 = Ellipsoid.wgs84().calculate_parametric_latitude(lat_1)
    beta2 = Ellipsoid.wgs84().calculate_parametric_latitude(lat_2)
    azim_aux = calculate_gc_azimuth(beta1, beta2, delta_long).to_degrees().v()
    delta_aux[i] = np.abs(azim_aux - row[Column.azimuth_1.value])
    
    i += 1

In [ ]:
stats.describe(delta_gc)

In [ ]:
stats.describe(delta_aux)

## Conclusion

The initial azimuth of a great circle on an auxiliary sphere is slightly closer to the initial azimuth of geodesic
than great circle initial azimuth using Karney's "random" test data.